In [2]:
import numpy as np
import itertools
import random
from statistics import stdev
import math
import ast

In [3]:
p_running = 0.9
p_failure = 1 - p_running
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v2/'

In [4]:
N = [50, 100, 200, 300, 400, 500]
N = [1024]
batches = 30
Products = 3
Market = 29
batches = 30
text_file = open(path + 'Evaluation_Set.txt', "r")
ls = text_file.read().split('\n')[:-1]
Scenarios = list(map(lambda x: ast.literal_eval(x), ls))
Probabilities = np.loadtxt(path + "p_scen.txt")


In [5]:
p1_mean = 137000
p2_mean = 135000
p3_mean = 133000

p1_sd = 2650
p2_sd = 8125
p3_sd = 4000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

In [6]:
np.random.seed(1)
demand = np.zeros((Products, Market))
demand[0] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
demand[1] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
demand[2] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))
#np.savetxt("/home/dkabe/Model_brainstorming/SAA_Analysis_v2/demand.txt", demand)

In [10]:
random.seed(1)
for num_scen in N:
    for batch in range(batches):
        indices = random.sample(range(len(Scenarios)), num_scen)
        sample_scenarios = [Scenarios[index] for index in indices]
        p_sample_scenarios = [Probabilities[index] for index in indices]
        p_factor = 1/np.sum(p_sample_scenarios)
        p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))  
        with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w') as f:
            for item in sample_scenarios:
                f.write("%s\n" % item)
        f.close()
        np.savetxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt', p_sample_scenarios)



In [9]:
np.sum(p_sample_scenarios)

1.0

In [8]:
# version 1
p_scen = np.loadtxt("/home/dkabe/Model_brainstorming/SAA_Analysis_v2/p_scen.txt")
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v2/'
scenarios = [50, 100, 200, 300, 400, 500]

for scen in scenarios:
    gaps = []
    ub_averages = []
    ub_max = []
    objvals_lb = np.loadtxt(path + "Objectives/" + str(scen) + '_results.txt')
    avg_lb = (np.mean(objvals_lb))
    std_lb = (stdev(objvals_lb))
    lbwidth = std_lb/math.sqrt(batches)*2.045
    lbmin = round(avg_lb - lbwidth,2)
    #print(avg_lb, lbmin)
    #print('ci on lower bound = [', round(avg_lb-lbwidth,2), ',', round(avg_lb+lbwidth,2), ']')
    for batch in range(batches):
        objvals_ub = np.loadtxt(path + "Upper_Bounds/" + str(scen) + "_scenarios/" + str(scen) + "_" + str(batch) + "_UB_results.txt")
        avg_ub = (np.sum(p_scen*objvals_ub))
        std_ub = math.sqrt(np.sum(p_scen*((objvals_ub - avg_ub)**2)))
        ubwidth = std_ub/math.sqrt(1024)*1.96
        ubmax = round(avg_ub + ubwidth, 2)
        gap = round(100*((ubmax - lbmin)/lbmin),2)
        ub_averages.append(avg_ub)
        gaps.append(gap)
        ub_max.append(ubmax)
    min_gap = min(gaps)
    index = gaps.index(min_gap)
    #print(ub_averages[index], ub_max[index], min_gap)
    print(avg_lb, "\t", ub_averages[index], "\t", lbmin, ub_max[index])

19362793.239666667 	 18935672.300249435 	 18273289.84 19317928.52
19771577.093999997 	 18935672.300249435 	 18887054.46 19317928.52
18967903.653333332 	 18935672.300249435 	 18089671.66 19317928.52
19080771.92066667 	 18935672.300249435 	 18552849.33 19317928.52
19169839.812 	 18935672.300249435 	 18746646.12 19317928.52
18931530.18066667 	 18935672.300249435 	 18609097.77 19317928.52
